In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e18/sample_submission.csv
/kaggle/input/playground-series-s3e18/train.csv
/kaggle/input/playground-series-s3e18/test.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s3e18/train.csv')
train.head()

,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,...,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2,EC3,EC4,EC5,EC6
0,0,323.390782,9.879918,5.875576,5.875576,4.304757,4.304757,2.754513,1.749203,0.000000,...,4.794537,35.527357,0,0,1,1,0,0,0,0
1,1,273.723798,7.259037,4.441467,5.834958,3.285046,4.485235,2.201375,1.289775,45.135471,...,13.825658,44.707310,0,0,0,1,1,0,0,0
2,2,521.643822,10.911303,8.527859,11.050864,6.665291,9.519706,5.824822,1.770579,15.645394,...,17.964475,45.660120,0,0,1,1,0,0,1,0
3,3,567.431166,12.453343,7.089119,12.833709,6.478023,10.978151,7.914542,3.067181,95.639554,...,31.961948,87.509997,0,0,1,1,0,0,0,0
4,4,112.770735,4.414719,2.866236,2.866236,1.875634,1.875634,1.036450,0.727664,17.980451,...,9.589074,33.333333,2,2,1,0,1,1,1,0


In [4]:
train.shape

(14838, 38)

In [5]:
train.describe()

,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,...,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2,EC3,EC4,EC5,EC6
count,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,...,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000,14838.000000
mean,7418.500000,515.153604,9.135189,5.854307,6.738497,4.432570,5.253221,3.418749,1.773472,29.202823,...,13.636941,49.309959,0.458215,0.459226,0.667745,0.798962,0.313789,0.279081,0.144831,0.151570
std,4283.505982,542.456370,6.819989,4.647064,5.866444,3.760516,4.925065,3.436208,1.865898,31.728679,...,14.598554,29.174824,0.667948,0.668111,0.471038,0.400790,0.464047,0.448562,0.351942,0.358616
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-5.430556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3709.250000,149.103601,4.680739,2.844556,2.932842,1.949719,2.034468,1.160763,0.503897,5.969305,...,4.794537,30.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,7418.500000,290.987941,6.485270,4.052701,4.392859,2.970427,3.242775,1.948613,1.073261,17.353601,...,9.589074,41.666667,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,11127.750000,652.652585,11.170477,7.486791,8.527859,5.788793,6.609350,4.502070,2.534281,44.876559,...,14.912664,56.090650,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,14837.000000,4069.959780,69.551167,50.174588,53.431954,32.195368,34.579313,22.880836,16.072810,363.705954,...,115.406157,384.450519,8.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
y1 = train.EC1
y1.head()

0    1
1    0
2    1
3    1
4    1
Name: EC1, dtype: int64

In [7]:
y2 = train.EC2
y2.head()

0    1
1    1
2    1
3    1
4    0
Name: EC2, dtype: int64

In [8]:
train.head()

,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,...,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2,EC3,EC4,EC5,EC6
0,0,323.390782,9.879918,5.875576,5.875576,4.304757,4.304757,2.754513,1.749203,0.000000,...,4.794537,35.527357,0,0,1,1,0,0,0,0
1,1,273.723798,7.259037,4.441467,5.834958,3.285046,4.485235,2.201375,1.289775,45.135471,...,13.825658,44.707310,0,0,0,1,1,0,0,0
2,2,521.643822,10.911303,8.527859,11.050864,6.665291,9.519706,5.824822,1.770579,15.645394,...,17.964475,45.660120,0,0,1,1,0,0,1,0
3,3,567.431166,12.453343,7.089119,12.833709,6.478023,10.978151,7.914542,3.067181,95.639554,...,31.961948,87.509997,0,0,1,1,0,0,0,0
4,4,112.770735,4.414719,2.866236,2.866236,1.875634,1.875634,1.036450,0.727664,17.980451,...,9.589074,33.333333,2,2,1,0,1,1,1,0


In [9]:
 train.drop(['EC1','EC2','EC3','EC4','EC5','EC6'], axis=1, inplace=True)


In [10]:
y1.value_counts(),y2.value_counts()

(1    9908
 0    4930
 Name: EC1, dtype: int64,
 1    11855
 0     2983
 Name: EC2, dtype: int64)

In [11]:
train_test_split(train,y1,)

[          id      BertzCT       Chi1      Chi1n      Chi1v      Chi2n  \
 3105    3105   551.179507   8.637535   5.336107   8.858718   4.582563   
 3148    3148    83.108030   3.553418   2.601938   2.601938   1.978458   
 13482  13482   201.871394   5.609061   2.316497   2.316497   1.962540   
 13240  13240   133.774438   3.180739   1.673397   1.673397   1.126913   
 10030  10030   133.774438   4.180739   2.210075   2.210075   1.345810   
 ...      ...          ...        ...        ...        ...        ...   
 5444    5444   299.171248   7.270857   4.657448   5.910092   2.762572   
 1343    1343   579.245113   7.613392   4.435575   4.435575   3.084798   
 12130  12130   802.439177  10.810968   6.913923   6.913923   4.526518   
 62        62  1131.608133  22.776672  13.944394  20.990690  10.551417   
 10175  10175   242.564575   7.613392   3.933552   3.933552   2.202381   
 
            Chi2v      Chi3v     Chi4n  EState_VSA1  ...  PEOE_VSA14  \
 3105    7.113245   5.057852  2.287209

In [12]:
categorical_cols = [cname for cname in train.columns if
                    train[cname].nunique() < 10 and 
                    train[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in train.columns if 
                train[cname].dtype in ['int64', 'float64']]

In [13]:
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


In [14]:
my_cols = categorical_cols + numerical_cols
X_train = train[my_cols].copy()
# X_valid = X_valid_full[my_cols].copy()

In [15]:
X_train.shape

(14838, 32)

# Logistic Regression

In [16]:
model = LogisticRegression(random_state=42)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

In [17]:
clf.fit(X_train,y1)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['id', 'BertzCT', 'Chi1',
                                                   'Chi1n', 'Chi1v', 'Chi2n',
                                                   'Chi2v', 'Chi3v', 'Chi4n',
                                                   'EState_VSA1', 'EState_VSA2',
                                                   'ExactMolWt',
                                                   'FpDensityMorgan1',
                                                   'FpDensityMorgan2',
                                                   'FpDensityMorgan3',
                                                   'HallKierAlpha',
                                                   'HeavyAtomMolWt', 'Kappa3',
                                                   'MaxAbsEStateIndex',
                                                   'MinEStateIndex',
                                                   'NumHeteroatoms',
                                                   'PEOE_VSA10', 'PEOE_VSA14',
                                                   'PEOE_VSA6', 'PEOE_VSA7',
                                                   'PEOE_VSA8', 'SMR_VSA10',
                                                   'SMR_VSA5', 'SlogP_VSA3',
                                                   'VSA_EState9', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('model', LogisticRegression(random_state=42))])

In [18]:
test = pd.read_csv('/kaggle/input/playground-series-s3e18/test.csv')
test.head()

,id,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,...,PEOE_VSA14,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2
0,14838,344.632371,7.283603,4.473966,5.834958,3.412257,4.651530,2.096558,1.116433,49.458581,...,13.512441,0.000000,0.000000,0.000000,26.809272,24.539800,4.794537,47.304082,1,1
1,14839,1432.410201,10.663869,7.079026,8.065215,5.297097,5.297097,3.924155,2.569694,0.000000,...,0.000000,34.947374,98.323987,9.606882,0.000000,53.378235,0.000000,43.166667,0,0
2,14840,83.352608,3.931852,1.774215,1.774215,1.073446,1.073446,0.467830,0.170838,5.969305,...,5.969305,0.000000,0.000000,6.420822,11.752550,13.344559,9.589074,24.666667,1,1
3,14841,150.255712,5.912790,3.548812,3.548812,2.595128,2.595128,1.642813,0.694113,0.000000,...,59.935299,0.000000,0.000000,0.000000,17.744066,32.290168,4.794537,26.778866,0,0
4,14842,1817.276351,24.910940,15.540529,20.047314,12.535886,17.730988,11.979618,4.431173,84.554972,...,23.468091,25.609359,0.000000,37.099000,69.141353,38.704130,50.697492,102.583333,0,0


In [19]:
pred1 = clf.predict(test)
pred1

array([1, 1, 1, ..., 1, 1, 1])

In [20]:
clf1 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

In [21]:
clf1.fit(X_train,y2)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['id', 'BertzCT', 'Chi1',
                                                   'Chi1n', 'Chi1v', 'Chi2n',
                                                   'Chi2v', 'Chi3v', 'Chi4n',
                                                   'EState_VSA1', 'EState_VSA2',
                                                   'ExactMolWt',
                                                   'FpDensityMorgan1',
                                                   'FpDensityMorgan2',
                                                   'FpDensityMorgan3',
                                                   'HallKierAlpha',
                                                   'HeavyAtomMolWt', 'Kappa3',
                                                   'MaxAbsEStateIndex',
                                                   'MinEStateIndex',
                                                   'NumHeteroatoms',
                                                   'PEOE_VSA10', 'PEOE_VSA14',
                                                   'PEOE_VSA6', 'PEOE_VSA7',
                                                   'PEOE_VSA8', 'SMR_VSA10',
                                                   'SMR_VSA5', 'SlogP_VSA3',
                                                   'VSA_EState9', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('model', LogisticRegression(random_state=42))])

In [22]:
pred2 = clf1.predict(test)
pred2

array([1, 1, 1, ..., 1, 1, 1])

# Random Forest

In [23]:
model = RandomForestClassifier(random_state=42)
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

In [24]:
clf.fit(X_train,y1)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['id', 'BertzCT', 'Chi1',
                                                   'Chi1n', 'Chi1v', 'Chi2n',
                                                   'Chi2v', 'Chi3v', 'Chi4n',
                                                   'EState_VSA1', 'EState_VSA2',
                                                   'ExactMolWt',
                                                   'FpDensityMorgan1',
                                                   'FpDensityMorgan2',
                                                   'FpDensityMorgan3',
                                                   'HallKierAlpha',
                                                   'HeavyAtomMolWt', 'Kappa3',
                                                   'MaxAbsEStateIndex',
                                                   'MinEStateIndex',
                                                   'NumHeteroatoms',
                                                   'PEOE_VSA10', 'PEOE_VSA14',
                                                   'PEOE_VSA6', 'PEOE_VSA7',
                                                   'PEOE_VSA8', 'SMR_VSA10',
                                                   'SMR_VSA5', 'SlogP_VSA3',
                                                   'VSA_EState9', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('model', RandomForestClassifier(random_state=42))])

In [25]:
pred1 = clf.predict(test)
pred1

array([1, 1, 1, ..., 0, 0, 0])

In [26]:
clf1 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

In [27]:
clf1.fit(X_train,y2)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['id', 'BertzCT', 'Chi1',
                                                   'Chi1n', 'Chi1v', 'Chi2n',
                                                   'Chi2v', 'Chi3v', 'Chi4n',
                                                   'EState_VSA1', 'EState_VSA2',
                                                   'ExactMolWt',
                                                   'FpDensityMorgan1',
                                                   'FpDensityMorgan2',
                                                   'FpDensityMorgan3',
                                                   'HallKierAlpha',
                                                   'HeavyAtomMolWt', 'Kappa3',
                                                   'MaxAbsEStateIndex',
                                                   'MinEStateIndex',
                                                   'NumHeteroatoms',
                                                   'PEOE_VSA10', 'PEOE_VSA14',
                                                   'PEOE_VSA6', 'PEOE_VSA7',
                                                   'PEOE_VSA8', 'SMR_VSA10',
                                                   'SMR_VSA5', 'SlogP_VSA3',
                                                   'VSA_EState9', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('model', RandomForestClassifier(random_state=42))])

In [28]:
pred2 = clf1.predict(test)
pred2

array([1, 1, 1, ..., 1, 1, 1])

# Decision Tree

In [29]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=42)
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])
clf1 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])


In [30]:
clf.fit(X_train,y1)
clf1.fit(X_train,y2)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['id', 'BertzCT', 'Chi1',
                                                   'Chi1n', 'Chi1v', 'Chi2n',
                                                   'Chi2v', 'Chi3v', 'Chi4n',
                                                   'EState_VSA1', 'EState_VSA2',
                                                   'ExactMolWt',
                                                   'FpDensityMorgan1',
                                                   'FpDensityMorgan2',
                                                   'FpDensityMorgan3',
                                                   'HallKierAlpha',
                                                   'HeavyAtomMolWt', 'Kappa3',
                                                   'MaxAbsEStateIndex',
                                                   'MinEStateIndex',
                                                   'NumHeteroatoms',
                                                   'PEOE_VSA10', 'PEOE_VSA14',
                                                   'PEOE_VSA6', 'PEOE_VSA7',
                                                   'PEOE_VSA8', 'SMR_VSA10',
                                                   'SMR_VSA5', 'SlogP_VSA3',
                                                   'VSA_EState9', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('model', DecisionTreeClassifier(random_state=42))])

In [31]:
pred1 = clf.predict(test)
pred1

array([0, 1, 1, ..., 1, 1, 1])

In [32]:
pred2 = clf1.predict(test)
pred2

array([0, 1, 1, ..., 1, 1, 1])

In [33]:

output1 = pd.DataFrame({'id': test.id,
                       'EC1': pred1,
                       'EC2': pred2})
output1.to_csv('submission3.csv', index=False)

# SAMPLE

In [34]:
sample = pd.read_csv('/kaggle/input/playground-series-s3e18/sample_submission.csv')
sample.head()

,id,EC1,EC2
0,14838,0.5,0.5
1,14839,0.5,0.5
2,14840,0.5,0.5
3,14841,0.5,0.5
4,14842,0.5,0.5


In [35]:

output1 = pd.DataFrame({'id': test.id,
                       'EC1': pred1,
                       'EC2': pred2})
output1.to_csv('submission1.csv', index=False)

In [36]:

output1 = pd.DataFrame({'id': test.id,
                       'EC1': pred1,
                       'EC2': pred2})
output1.to_csv('submission2.csv', index=False)